In [2]:
import torch.nn as nn
import torch

- Sử dụng `CausalAttention` được implement ở phần trước.

In [ ]:
class CausalAttention(nn.Module):
    def __init__(self, dim_in, dim_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.dim_out = dim_out
        self.W_query = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_key = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_value = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(   # Đăng ký một tensor không phải là tham số của mô hình
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, dim_in = x.shape

        # 3 Q,K,V có shape (batch_size, context_length, dim_out)
        keys = self.W_key(x)    
        queries = self.W_query(x)
        values = self.W_value(x)

        # keys.transpose(1, 2) đổi shape từ (B, T, D) thành (B, D, T)
        attention_scores = queries @ keys.transpose(1, 2)  
        # [:num_tokens, :num_tokens] -- Slicing mask để phù hợp với số token hiện tại
        attention_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf  # Dùng được từ register_buffer 
        )
        
        attention_weights = torch.softmax(
            attention_scores / keys.shape[-1]**0.5, dim=-1
        )
        attention_weights = self.dropout(attention_weights)

        context_vectors = attention_weights @ values
        
        print("Context vector:")
        print(context_vectors)

        return context_vectors

- Wrapper class triển khai `multi-head attention`.

In [4]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, dim_in, dim_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(dim_in, dim_out, context_length, dropout) for _ in range(num_heads)]
        )

    def forward(self, x):
        # dim=-1 để concat theo chiều cuối cùng, do mỗi head output shape (batch_size, num_tokens, dim_out)
        # dim_out là chiều của mỗi context vector
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [13]:
torch.manual_seed(123)
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)
batch = torch.stack([inputs, inputs], dim=0)  # Tạo batch size = 2
context_length = batch.shape[1]   # Num_tokens
dim_in, dim_out = batch.shape[2], 2
multi_head_attn = MultiHeadAttentionWrapper(
    dim_in=dim_in,
    dim_out=dim_out,
    context_length=context_length,
    dropout=0.0,
    num_heads=2,
)
context_vectors = multi_head_attn(batch)

print("\nMerged context vector:")
print(context_vectors)
print(f"context_vectors.shape: {context_vectors.shape} -- (batch_size, context_length / num_tokens, dim_out * num_heads)")

Context vector:
tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
Context vector:
tensor([[[0.4772, 0.1063],
         [0.5891, 0.3257],
         [0.6202, 0.3860],
         [0.5478, 0.3589],
         [0.5321, 0.3428],
         [0.5077, 0.3493]],

        [[0.4772, 0.1063],
         [0.5891, 0.3257],
         [0.6202, 0.3860],
         [0.5478, 0.3589],
         [0.5321, 0.3428],
         [0.5077, 0.3493]]], grad_fn=<UnsafeViewBackward0>)

Merged context vector:
tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5

- Input có _2 batch_, _batch 1 & batch 2_ giống nhau, _weights head 1 & weights head 2_ khác nhau nên như output _context vector_ của head 1 & head 2 cũng khác nhau. Còn cái bạn thấy giống nhau ở _merged context vector_ là output của 2 _batch_, và vì _batch_ giống nhau nên output cũng giống nhau.

- Vẫn giữ nguyên `num_heads`, giờ ta sẽ thử thay đổi để _merged context vector_ chỉ có dim=2 thay vì như cũ là _4_.

In [16]:
dim_out = 1
multi_head_attn = MultiHeadAttentionWrapper(
    dim_in=dim_in,
    dim_out=dim_out,
    context_length=context_length,
    dropout=0.0,
    num_heads=2,
)
context_vectors = multi_head_attn(batch)

print("\nMerged context vector:")
print(context_vectors)
print(f"context_vectors.shape: {context_vectors.shape} -- (batch_size, context_length / num_tokens, dim_out * num_heads)")

Context vector:
tensor([[[0.0189],
         [0.2181],
         [0.2804],
         [0.2830],
         [0.2476],
         [0.2748]],

        [[0.0189],
         [0.2181],
         [0.2804],
         [0.2830],
         [0.2476],
         [0.2748]]], grad_fn=<UnsafeViewBackward0>)
Context vector:
tensor([[[0.2729],
         [0.3037],
         [0.3125],
         [0.2793],
         [0.2541],
         [0.2513]],

        [[0.2729],
         [0.3037],
         [0.3125],
         [0.2793],
         [0.2541],
         [0.2513]]], grad_fn=<UnsafeViewBackward0>)

Merged context vector:
tensor([[[0.0189, 0.2729],
         [0.2181, 0.3037],
         [0.2804, 0.3125],
         [0.2830, 0.2793],
         [0.2476, 0.2541],
         [0.2748, 0.2513]],

        [[0.0189, 0.2729],
         [0.2181, 0.3037],
         [0.2804, 0.3125],
         [0.2830, 0.2793],
         [0.2476, 0.2541],
         [0.2748, 0.2513]]], grad_fn=<CatBackward0>)
context_vectors.shape: torch.Size([2, 6, 2]) -- (batch_size, conte

- Final `Multi-Head Attention` class

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, dim_in, dim_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert dim_out % num_heads == 0, "dim_out must be divisible by num_heads"
        self.dim_out = dim_out,
        self.num_heads = num_heads
        self.head_dim = dim_out // num_heads
        self.W_query = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_key = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_value = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.out_proj = nn.Linear(dim_out, dim_out)         # Output Projection
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
        
    def forward(self, x):
        batch, num_tokens, dim_in = x.shape 
        keys = self.W_key(x)
        values = self.W_value(x)
        queries = self.W_query(x)
        
        # View
        keys = keys.view(batch, num_tokens, self.num_heads, self.head_dim)
        values = values.view(batch, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(batch, num_tokens, self.num_heads, self.head_dim)
        
        # Transpose from (batch, num_tokens, num_heads, head_dim) to (batch, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        values = values.transpose(1, 2)
        queries = queries.transpose(1, 2)
        
        # Compute attention scores
        attention_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        
        attention_scores.masked_fill_(mask_bool, -torch.inf)
        
        attention_weights = torch.softmax(
            attention_scores / keys.shape[-1]**0.5, dim=-1
        )
        attention_weights = self.dropout(attention_weights)
        
        # Compute context vector
        context_vector = (attention_weights @ values).transpose(1, 2)
        
        context_vector = context_vector.contiguous().view(batch, num_tokens, self.dim_out)
        context_vector = self.out_proj(context_vector)
        
        return context_vector

- Điểm đầu tiên khác với _implement class_ trước là `.view()`.

- `.view()` sẽ split vector thành các đoạn nhỏ hơn bằng nhau. Ví dụ:

``` python
        import torch

        vector = torch.tensor([[1, 2, 3, 4]])     # shape (1, 4)
        vector = vector.view(2, 2)      # (2, 2) tương ứng với num_heads & head_dim trong MultiHeadAttention
        print(vector)  # shape (2, 2)

        tensor([[1, 2],
                [3, 4]])
```

- Mục đích của `.view()` là để tách vector thành các _head_ nhỏ hơn, mỗi _head_ sẽ trích xuất được các đặc trưng khác nhau.

- Để dễ hình dung, ta có input là _tensor_ của 1 token, _tensor_ này sau khi đưa qua 3 ma trận q,k,v $W$ ta có 3 _tensor_ được biến đổi để tăng _số chiều_ lên rất nhiều. Từ đó, _tensor_ sẽ được chia nhỏ về các _head_ để train parallel.

- _.view()_ cũng có thể _merge_ các chiều, ví dụ tensor có shape (1, 2, 2, 3) có thể được merge thành (1, 4, 3).

- Điểm thứ hai là `.transpose()` sau khi _.view()_.

- Cùng xem ví dụ để dễ hình dung nhé:

``` python
        queries = [
            [1.1, 1.2, 2.1, 2.2], # "Your"
            [3.1, 3.2, 4.1, 4.2], # "journey"
            ...
        ]

        queries_viewed = [
            # Từ "Your"
            [
                [1.1, 1.2], # Của Head 1
                [2.1, 2.2]  # Của Head 2
            ],
            # Từ "journey"
            [
                [3.1, 3.2], # Của Head 1
                [4.1, 4.2]  # Của Head 2
            ],
            ...
        ]

        queries_transposed = [
            # Head 1
            [
                [1.1, 1.2],  # 1 phần từ "Your"
                [3.1, 3.2],  # 1 phần từ "journey"
                ...
            ],
            # Head 2
            [
                [2.1, 2.2],  # 1 phần từ "Your"
                [4.1, 4.2],  # 1 phần từ "journey"
                ...
            ],
        ]
``` 

- `.transpose()` sẽ hoán đổi chiều của _num-tokens và num-heads_ cho nhau để gom nhóm theo head. Có thể thấy rằng tại mỗi _head_ đều có danh sách _tensor_ của mọi _input tokens_, các _tensor_ này được tách ra từ _tensor_ gốc của _input tokens_ nhằm mục đích tìm được 1 đặc trưng nào đó.

- Có 1 ghi nhớ nhỏ là quy tắc `Batch Matrix Multiplication`. Pytorch coi các chiều phía trước (ngoài 2 chiều cuối) là các _batch_ riêng biệt và chỉ thực hiện nhân ma trận trên _2 chiều cuối cùng_.

- Như vậy tại bước tính `attention_scores`, ta vẫn tính theo cách thông thường bằng cách lấy _queries @ keys.T_, với _keys.T_ là hoán đổi trên 2 chiều cuối cùng _(num_tokens, head_dim)_, về lại bài toán của `CausalAttention`.

- Sau khi tính được _context_vector_, ta cần _transpose_ để về lại chiều _(batch, num-tokens, num-heads, head-dim)_, sau đó merge bằng _.view()_ để về lại chiều _(batch, num-tokens, dim-out)_ (trong đó `dim-out = num-heads * head-dim`).

- Thêm 1 ghi nhớ nhỏ nữa là các hàm như `.transpose()`, `.expand()`, ... đều không di chuyển dữ liệu trong RAM, mà chỉ thay đổi metadata index của tensor. Vì vậy, sau khi `.transpose()` thì metadata index bị thay đổi và `.view()` ngay sau đó thì sẽ lỗi vì `.view()` thích xử lí đơn giản theo vị trí trên RAM, không muốn duyệt theo metadata index kia. Cho nên sau khi `.transpose()` cần `.continuous()` để sắp xếp lại vị trí trên RAM. 

- Sau khi `.view()` để ghép _tensor_ của các _head_ thì đưa qua một lớp `nn.Linear()` (Output Projection), là 1 phép nhân ma trận để trộn đều thông tin từ _các heads_ với nhau thành 1 _tensor_ tổng hợp. Bước này không bắt buộc về mặt toán học, nhưng hầu hết các LLM hiện đại đều dùng.